读取数据

In [1]:
import pandas as pd

In [2]:
train_json = pd.read_json('../train_dataset/train.json')

In [3]:
train_json['annotations'][0]

{'filename': 'train_images\\00001.jpg',
 'period': 'Morning',
 'weather': 'Cloudy'}

In [4]:
train_json['filename'] = train_json['annotations'].apply(lambda x : x['filename'].replace('\\', '/'))
train_json['period'] = train_json['annotations'].apply(lambda x : x['period'])
train_json['weather'] = train_json['annotations'].apply(lambda x : x['weather'])


In [5]:
train_json.head()

,annotations,filename,period,weather
0,"{'filename': 'train_images\00001.jpg', 'period...",train_images/00001.jpg,Morning,Cloudy
1,"{'filename': 'train_images\00002.jpg', 'period...",train_images/00002.jpg,Afternoon,Cloudy
2,"{'filename': 'train_images\00003.jpg', 'period...",train_images/00003.jpg,Morning,Cloudy
3,"{'filename': 'train_images\00004.jpg', 'period...",train_images/00004.jpg,Morning,Sunny
4,"{'filename': 'train_images\00005.jpg', 'period...",train_images/00005.jpg,Afternoon,Cloudy


标签编码

In [6]:
train_json['period'], period_dict = pd.factorize(train_json['period'])
train_json['weather'], weather_dict = pd.factorize(train_json['weather'])


In [7]:
period_dict,weather_dict

(Index(['Morning', 'Afternoon', 'Dawn', 'Dusk'], dtype='object'),
 Index(['Cloudy', 'Sunny', 'Rainy'], dtype='object'))

统计标签

In [8]:
train_json['period'].value_counts()

0    1613
1     829
3     124
2      34
Name: period, dtype: int64

In [9]:
train_json['weather'].value_counts()

0    1119
1     886
2     595
Name: weather, dtype: int64

In [10]:
from torchvision import transforms as T
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import numpy as np

In [11]:
class WeatherDataset(Dataset):
    def __init__(self, df):
        super(WeatherDataset, self).__init__()
        self.df = df

        self.transform = T.Compose([
            T.Resize(size=(340, 340)),
            T.RandomCrop(size=(224, 224)),
            T.RandomRotation(10),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.ToTensor(),
            T.Normalize((0.5,), (0.5,))
        ])

    def __getitem__(self, index):
        file_name = '../train_dataset/' + self.df['filename'].iloc[index]
        img = Image.open(file_name)
        img = self.transform(img)
        return img, torch.tensor(self.df['period'].iloc[index]), torch.tensor(self.df['weather'].iloc[index])
    def __len__(self):
        return len(self.df)

In [12]:
train_dataset = WeatherDataset(train_json.iloc[:-500])
val_dataset = WeatherDataset(train_json.iloc[-500:])

In [13]:
i = 0
for x, y1, y1 in train_dataset:
    i+=1
    print(x.shape)
    if i == 10:
        break

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [14]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)

模型搭建

In [15]:
class WeatherModel(nn.Module):
    def __init__(self):
        super(WeatherModel, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6400, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
        self.fc1 = nn.Linear(10, 4)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        out = self.conv(x)
        fc = self.fc(out)
        logist1 = self.fc1(fc)
        logist2 = self.fc2(fc)
        return logist1, logist2

In [16]:
model = WeatherModel()
model(torch.tensor(np.random.rand(10, 3, 224, 224).astype(np.float32)))

(tensor([[ 0.2707, -0.2762, -0.0278, -0.1123],
         [ 0.2885, -0.2712, -0.0255, -0.1009],
         [ 0.2854, -0.2857, -0.0331, -0.0960],
         [ 0.2811, -0.2723, -0.0313, -0.1082],
         [ 0.2783, -0.2835, -0.0269, -0.1069],
         [ 0.2784, -0.2761, -0.0376, -0.1103],
         [ 0.2889, -0.2762, -0.0260, -0.0970],
         [ 0.2858, -0.2709, -0.0386, -0.1094],
         [ 0.2827, -0.2724, -0.0277, -0.1019],
         [ 0.2791, -0.2707, -0.0371, -0.1077]], grad_fn=<AddmmBackward>),
 tensor([[-0.2941, -0.0799, -0.0146],
         [-0.3071, -0.0796, -0.0158],
         [-0.2934, -0.0764, -0.0115],
         [-0.3006, -0.0781, -0.0192],
         [-0.2974, -0.0749, -0.0118],
         [-0.2957, -0.0690, -0.0204],
         [-0.2973, -0.0836, -0.0013],
         [-0.2935, -0.0756, -0.0145],
         [-0.3009, -0.0829, -0.0134],
         [-0.2927, -0.0714, -0.0173]], grad_fn=<AddmmBackward>))

模型训练

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [19]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
for epoch in range(10):
    train_loss, val_loss = [], []
    train_acc1, train_acc2 = [], []
    val_acc1, val_acc2 = [], []

    model.train()
    for i, (x, y1, y2) in enumerate(train_loader):
        x = x.to(device)
        y1 = y1.to(device)
        y2 = y2.to(device)
        optimizer.zero_grad()
        pred1, pred2 = model(x)
        loss = criterion(pred1, y1) + criterion(pred2, y2)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        train_acc1.append((pred1.argmax(1) == y1.flatten()).cpu().numpy().mean())
        train_acc2.append((pred2.argmax(1) == y2.flatten()).cpu().numpy().mean())

    model.eval()
    for i,(x, y1, y2) in enumerate(val_loader):
        x = x.to(device)
        y1 = y1.to(device)
        y2 = y2.to(device)
        pred1, pred2 = model(x)
        loss = criterion(pred1, y1) + criterion(pred2, y2)
        val_loss.append(loss.item())
        val_acc1.append((pred1.argmax(1) == y1.flatten()).cpu().numpy().mean())
        val_acc2.append((pred2.argmax(1) == y2.flatten()).cpu().numpy().mean())

    if epoch % 1 == 0:
        print(f'\nEpoch:{epoch}')
        print(f'Loss:{np.mean(train_loss):3.5f}/{np.mean(val_loss):3.5f}')
        print(f'period acc:{np.mean(train_acc1):3.5f}/{np.mean(val_acc1):3.5f}')
        print(f'weather acc:{np.mean(train_acc2):3.5f}/{np.mean(val_acc2):3.5f}')


Epoch:0
Loss:1.76751/2.00032
period acc:0.62524/0.59600
weather acc:0.61238/0.33800

Epoch:1
Loss:1.63037/1.74520
period acc:0.62286/0.59600
weather acc:0.70429/0.55600

Epoch:2
Loss:1.52659/1.87893
period acc:0.62333/0.59600
weather acc:0.73619/0.49400

Epoch:3
Loss:1.46820/2.20197
period acc:0.62571/0.59600
weather acc:0.73762/0.46600

Epoch:4
Loss:1.43044/2.03916
period acc:0.63476/0.56800
weather acc:0.75714/0.49400

Epoch:5
Loss:1.40575/2.44580
period acc:0.63429/0.58200
weather acc:0.76667/0.43600

Epoch:6
Loss:1.35225/1.95673
period acc:0.64810/0.59600
weather acc:0.77810/0.58800

Epoch:7
Loss:1.30548/2.05118
period acc:0.65905/0.59800
weather acc:0.78286/0.46000

Epoch:8
Loss:1.28649/1.57838
period acc:0.66810/0.64400
weather acc:0.78714/0.66600

Epoch:9
Loss:1.21244/1.57354
period acc:0.68190/0.67600
weather acc:0.79190/0.62600


In [20]:
path = 'alexnet.pth'
torch.save(model.state_dict(), path)
